In [ ]:
# Librearía de hugging face donde se encuentra el modelo preentrenado
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from google.colab import drive
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Otorgar acceso a drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
model = AutoModel.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.we

In [ ]:

def get_embedding(text):
    # Tokenize and convert to input IDs
    inputs = tokenizer.encode_plus(text, return_tensors='pt', padding='max_length', truncation=True, max_length=128)
    # Forward pass, get hidden states
    outputs = model(**inputs)
    # Use the [CLS] embedding as the sentence embedding
    embeddings = outputs.last_hidden_state[:, 0, :]
    return embeddings

In [ ]:
folder = '/content/drive/MyDrive/Manpower/Homologacion cargos/Production/data'

df_muestra = pd.read_excel(f"{folder}/Muestra.xlsx")
df_ocupaciones = pd.read_excel(f"{folder}/BaseCUOC.xlsx", sheet_name="Ocupacion")
df_descripciones = pd.read_excel(f"{folder}/BaseCUOC.xlsx", sheet_name="Descripcion")


In [ ]:
# Codificar en embeddings todas las ocupaciones. Se concatena el título y la descripción y se halla el embedding
ocupaciones = list()
for i in range(len(df_ocupaciones)):
  # Las tablas de Nombre ocupacion y descripción están sincronizadas en el excel
  titulo = df_ocupaciones.loc[i, "Nombre Ocupación"]
  descripcion = df_descripciones.loc[i, "Descripción Ocupación"]
  concatenado = (titulo+". "+descripcion).lower()
  ocupaciones.append({
      "id" : df_ocupaciones.loc[i, "Ocupación"],
      "titulo" : titulo,
      "descripcion" : descripcion,
      "concatenado" : concatenado, 
      "procesado" : get_embedding(concatenado)
  })

In [ ]:
# Codificar en embeddings de los cargos de muestra. Se concatena el título y la descripción, y se halla el embedding
cargos = list()
for i in range(len(df_muestra)):
  titulo = df_muestra.loc[i,"Cargo"]
  descripcion = df_muestra.loc[i,"Descripción SP"]
  concatenado = (titulo+". " +descripcion).lower() 
  cargos.append({
          "titulo" : titulo,
          "descripcion" : descripcion,
          "concatenado" : concatenado,
          "procesado" : get_embedding(concatenado),
          "homologacion_manual" : { "id": df_muestra.loc[i, "Codigo Cuoc"], "titulo": df_muestra.loc[i, "Ocupación"]}
      })

In [ ]:

for cargo in cargos:
  cargo["similarities"] = list()
  for ocupacion in ocupaciones:
    cargo_vec = cargo["procesado"].detach().numpy()
    ocupacion_vec = ocupacion["procesado"].detach().numpy()
    sim = cosine_similarity(cargo_vec, ocupacion_vec)[0][0]
    cargo["similarities"].append((sim, ocupacion["id"], ocupacion["titulo"]))
  
  cargo["similarities"].sort(reverse=True)

In [ ]:
resultados = list()
for cargo in cargos:
  more_similar_oc_id = cargo["similarities"][0][1]
  similarity = cargo["similarities"][0][0]

  if more_similar_oc_id == cargo["homologacion_manual"]["id"]:
    correcto = True
  else:
    correcto = False
  
  resultados.append((cargo["titulo"], cargo["descripcion"], more_similar_oc_id, similarity, correcto))

column_names = ["titulo_cargo", "descripcion_cargo", "ocupacion_mas_similar", "similitud", "es_correcto"]
resultados = pd.DataFrame(resultados, columns=column_names)

In [ ]:
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import torch

# Load BETO
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
model = AutoModel.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

def get_embedding(text):
    # Tokenize and convert to input IDs
    inputs = tokenizer.encode_plus(text, return_tensors='pt', padding='max_length', truncation=True, max_length=128)
    # Forward pass, get hidden states
    outputs = model(**inputs)
    # Use the [CLS] embedding as the sentence embedding
    embeddings = outputs.last_hidden_state[:, 0, :]
    return embeddings

# Define two Spanish sentences
sentence1 = "Esto es una oración en español."
sentence2 = "Esta es otra oración en español."

# Get embeddings
embeddings1 = get_embedding(sentence1)
embeddings2 = get_embedding(sentence2)

# Compute Cosine Similarity
cos_sim = cosine_similarity(embeddings1.detach().numpy(), embeddings2.detach().numpy())[0][0]

print(f"Cosine Similarity: {cos_sim}")